# 9. Qt model view paradigm

Qt does not follow MVC, but it has a paradigm called model/view. It's similar to MVC, it just puts the business logic (normally in controller) into view. In one phrase, `model handles data and provide api to retrive and store data`, `the view handles user interacation, and how to render data in the interface`. To facilitate the management of user input, QT adds a concept called **delegate**, it can customize the data edition and rendering. Below figure shows the workflow.
![qt_model_view_paradigm.png](../../image/qt_model_view_paradigm.png)

The two standard workflow of the model view paradigm is:
1. data modification in model will trigger a signale, the signale will be captured by the view, then the view stats to refresh the interface
2. User interacts with the view, the view can ask model to fetch data

Qt provide an `abstract class for all three concept(e.g. model, view, delegate)`, which provides the basic interfaces and some implementation.

## Model abstract class
In QT all models extends **QAbstractItemModel**, note model may or may not store the actual data. The actual data may be in a file or database, the model may be just a wrapper of it.

Qt provides already some model implementation:
- QStringListModel： model for handling string list
- QStandardItemModel： model for handling hierarchical data such as Tree.
- QFileSystemModel： model for handling local file system
- QSortFilterProxyModel： this model is an overcoat of other model, it can provide sort and filtering service.
- QSqlQueryMode： model for handling sql query of db
- QSqlTableModel：model for handling table of a db
- QSqlRelationalTableModel： model for handling table relation


## View abstract class

In QT, all view extends **QAbstractItemView**, which provides a standard interface. Qt provides below view implementation:

- QListView：view for rendering list data
- QTreeView： view for rendering hierarchical data such as Tree
- QTableView：view for rendering tabular data
- QColumnView：it can use `QListView` to render hierarchical data such as Tree. Each level of the tree can be rendered by a `QListView`.
- QHeaderView： it renders the header of other view, for example, the column and row name of `QTableView`
- QUndoView: 继承自QListView，用于显示Undo列表信息;
- QListWidget: 继承自QListView，用于显示组件列表，它的项是QListWidgetItem对象;
- QTreeWidget: 继承自QTreeView, 按树状方式显示组件，它的项是QTreeWidgetItem对象；
- QTableWidget: 继承自QTableView, 按表格方式显示组件，它的项是QTableWidgetItem对象。

## Delegate abstract class

**QAbstractItemDelegate** is the abstract class of the `Delegate` concept. By default, if user does not provide a delegate implementation, QT will use **QStyledItemDelegate** as the default delegate.

QStyledItemDelegate和QItemDelegate是相互独立的用于实现视图（views）中items的描绘和编辑功能的方法。不同点在于QStyledItemDelegate使用当前的样式（style）来描绘items，因此，当我们实现定制委托或者使用Qt Style Sheets时，建议使用QStyledItemDelegate作为基类



## An example of model view

In below example, we create a filesystem model, then we associate the model to a tree view and a list view. So `one model can be used by multiple views`.

In [ ]:
import sys

from PyQt6.QtCore import QDir
from PyQt6.QtGui import QFileSystemModel
from PyQt6.QtWidgets import QWidget, QVBoxLayout, QSplitter, QTreeView, QListView, QApplication


class ModelViewDemo(QWidget):
    def __init__(self, parent=None):
        super(ModelViewDemo, self).__init__(parent)
        self.setWindowTitle("Model view paradigm demo")
        self.resize(480, 320)
        self.initUi()

    def initUi(self):
        layout = QVBoxLayout(self)
        splitter = QSplitter(self)

        # create a model
        fsm = QFileSystemModel()
        fsm.setRootPath(QDir.currentPath())

        # create a tree view of the model
        tv = QTreeView(splitter)
        tv.setModel(fsm)
        tv.setRootIndex(fsm.index(QDir.currentPath()))

        # create a list view of the model
        lv = QListView(splitter)
        lv.setModel(fsm)
        lv.setRootIndex(fsm.index(QDir.currentPath()))

        layout.addWidget(splitter)
        self.setLayout(layout)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window= ModelViewDemo()
    window.show()
    sys.exit(app.exec())


In [1]:
import sys
from PyQt6.QtWidgets import QApplication, QMainWindow, QPushButton, QVBoxLayout, QLabel,QWidget


## 9.1 Simple button example

As we explained before, the workflow
1. User trigger an event (mouse click, keyboard presses, etc.)
2. A widget catches the event (e.g. QPushButton, ) and emits a signal
3. A Signal can be handled by a slot
4. The triggered slot runs some related operation

Some of the most relevant features of signals and slots include the following:

- A signal can be connected to one or many slots.
- A signal may also be connected to another signal.
- A slot may be connected to one or many signals.
- Signals can also send data to provide additional context about what happened.
- Any function (or method) in your application can be used as a slot, simply by connecting the signal to it If the signal sends data, then the receiving function will receive that data too. Many Qt widgets also have their own built-in slots, meaning you can hook Qt widgets together directly

If we have many events, widgets and slots. The code will soon become unreadable. So we need to use a pattern to classify and modular code by their functionalities. PyQT uses MVC (Model, view, control) pattern.

In below example, we have a simple button, when users push the button, a `button.clicked` signale will be emitted. This signal will be connected with a slot called `theButtonWasClicked`.


In [2]:

def theButtonWasClicked():
    label = msgLabel
    history = label.text()
    if history:
        label.setText(f"{history}\nclicked!")
    else:
        label.setText("clicked")

app = QApplication([])
window = QWidget()
window.setWindowTitle("My App")
# setup Vertical layout manager
layout = QVBoxLayout()

# create button widget and add it to layout
button = QPushButton("Press Me!")
layout.addWidget(button)
# link button with function theButtonWasClicked
button.clicked.connect(theButtonWasClicked)

# create a text area and add it to layout
msgLabel = QLabel("")
layout.addWidget(msgLabel)

# Set the main Window layout.

window.setLayout(layout)



window.show()

sys.exit(app.exec())

qt.qpa.xcb: X server does not support XInput 2


SystemExit: 0

/home/pengfei/.cache/pypoetry/virtualenvs/python-desktop-app-framework-xY3vli_t-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Receiving data

We've heard already that `signals can also send data to slot`, which provides more information about what has just happened.

For the button, we can set it as `Checkable`, then the .clicked signal of the button can send a checked (or toggled) state for the button. By defaults, buttons are not checkable.

The state is passed as the second parameter of the function `_theButtonWasToggled`

In [2]:
class MainUI(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("My App")
        # setup Vertical layout manager
        layout = QVBoxLayout()
        # create button widget and add it to layout
        button = QPushButton("Press Me!")
        # make button checkable
        button.setCheckable(True)
        layout.addWidget(button)
        # create a text area and add it to layout
        self.msgLabel = QLabel("")
        layout.addWidget(self.msgLabel)
        # Set the main Window layout.
        self.setLayout(layout)
        # link the checked button with function
        button.clicked.connect(self._theButtonWasToggled)
    # this function takes an extra argument, so if you liked it with another button that does not send status
    # the value of checked will be always None
    def _theButtonWasToggled(self,checked):
        label = self.msgLabel
        history = label.text()
        if history:
            label.setText(f"{history}\nclicked! status {checked}")
        else:
            label.setText(f"clicked, status {checked}")

app = QApplication([])
mainUI=MainUI()
mainUI.show()

sys.exit(app.exec())

qt.qpa.xcb: X server does not support XInput 2


SystemExit: 0

/home/pengfei/.cache/pypoetry/virtualenvs/python-desktop-app-framework-xY3vli_t-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Storing data

Often it is useful to store the `current state of a widget in a Python variable`. This allows you to work with the values like any other Python variable and without accessing the original widget. You can either store these values as individual variables or use a dictionary if you prefer. In the next example we store the checked value of our button in a variable called buttonIsChecked on self.

In [4]:
class MainUI(QWidget):
    def __init__(self):
        super().__init__()
        # variable to store button current state
        self.buttonIsChecked = True
        self.setWindowTitle("My App")
        # setup Vertical layout manager
        layout = QVBoxLayout()
        # create button widget and add it to layout
        button = QPushButton("Press Me!")
        # make button checkable
        button.setCheckable(True)
        layout.addWidget(button)
        # create a text area and add it to layout
        self.msgLabel = QLabel("")
        layout.addWidget(self.msgLabel)
        # Set the main Window layout.
        self.setLayout(layout)
        button.clicked.connect(self._theButtonWasToggled)

    def _theButtonWasToggled(self,checked):
        self.buttonIsChecked=checked
        label = self.msgLabel
        history = label.text()
        if history:
            label.setText(f"{history}\nclicked! status {self.buttonIsChecked}")
        else:
            label.setText(f"clicked, status {self.buttonIsChecked}")

app = QApplication([])
mainUI=MainUI()
mainUI.show()

sys.exit(app.exec())

SystemExit: 0

/home/pengfei/.cache/pypoetry/virtualenvs/python-desktop-app-framework-xY3vli_t-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 9.3 Connecting widgets together directly

So far we've seen examples of connecting `widget signals to Python methods`. When a signal is fired from the widget, our Python method is called and receives the data from the signal. But you don't always need to use a Python function to handle signals. `You can also connect Qt widgets directly to another widget`.

In the following example, we add a `QLineEdit` widget and `a QLabel` to the window. In the `__init__` of the MainWindow class, we connect the `QlineEdit .textChanged signal` to the `QLabel.setText method (slot)` . Now any time the text changes in the QLineEdit the QLabel will receive that text to it's .setText method.

In [2]:
from PyQt6.QtWidgets import QLineEdit


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("My App")

        self.label = QLabel()

        self.input = QLineEdit()
        self.input.textChanged.connect(self.label.setText)

        layout = QVBoxLayout()
        layout.addWidget(self.input)
        layout.addWidget(self.label)

        container = QWidget()
        container.setLayout(layout)

        # Set the central widget of the Window.
        self.setCentralWidget(container)


app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()

qt.qpa.xcb: X server does not support XInput 2


0

> Most Qt widgets have slots available, to which you can connect any signal that emits the same type that it accepts. The widgets slot should be listed as "Public Slots". Otherwise, you shouldn't use it. The widget documentation can provide you more information. For example, see https://doc.qt.io/qt-5/qlabel.html#public-slots[QLabel].

## 9.4 Event

**Every interaction the user has with a Qt application is an event**. There are many types of event, each representing a different type of interaction. `Qt represents these events using event objects` which package up information about what happened. These `events are passed to specific event handlers on the widget where the interaction occurred`.

By defining custom, or extended event handlers you can alter the way your widgets respond to these events. Event handlers are defined just like any other method, but the name is specific for the type of event they handle.

One of the main events which widgets receive is the `QMouseEvent`. QMouseEvent events are created for each and every mouse movement and button click on a widget. The following event handlers are available for handling mouse events:

| Event handler         | 	Event type moved      |
|-----------------------|------------------------|
| mouseMoveEvent	       | Mouse moved            |
| mousePressEvent	      | Mouse button pressed   |
| mouseReleaseEvent     | 	Mouse button released |
| mouseDoubleClickEvent | 	Double click detected |

For example, clicking on a widget will cause a QMouseEvent to be sent to the .mousePressEvent event handler on that widget. This handler can use the event object to find out information about what happened, such as what triggered the event and where specifically it occurred.

You can intercept events by sub-classing and overriding the handler method on the class. You can choose to filter, modify, or ignore events, passing them up to the normal handler for the event by calling the parent class function with `super()`. These could be added to your main window class as follows. In each case `e` will receive the incoming event.

You'll notice that mouse move events are only registered when you have the button pressed down. You can change this by calling self.setMouseTracking(True) on the window. You may also notice that the press (click) and double-click events both fire when the button is pressed down. Only the release event fires when the button is released. Typically, to register a click from a user you should watch for both the mouse down and the release.

Inside the event handlers you have access to an event object. This object contains information about the event and can be used to respond differently depending on what exactly has occurred. We'll look at the mouse event objects next.

In [3]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.label = QLabel("Click in this window")
        self.setCentralWidget(self.label)

    def mouseMoveEvent(self, e):
        self.label.setText("mouseMoveEvent")

    def mousePressEvent(self, e):
        self.label.setText("mousePressEvent")

    def mouseReleaseEvent(self, e):
        self.label.setText("mouseReleaseEvent")

    def mouseDoubleClickEvent(self, e):
        self.label.setText("mouseDoubleClickEvent")


app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()

0

In [2]:

# control class
class Controller:
    def __init__(self, view, model=None):
        self._view=view
        self._model=model
        self._connectSignalsAndSlots()

    def _theButtonWasClicked(self):
        print("Clicked!")

    def _connectSignalsAndSlots(self):
        self._view.button.clicked.connect(self._theButtonWasClicked)


# view class
class MainView(QMainWindow):

    def __init__(self):
        super(MainView, self).__init__()

        self.setWindowTitle("My App")
        self.button = QPushButton("Press Me!")
        self.button.setCheckable(True)
        # Set the central widget of the Window.
        self.setCentralWidget(self.button)



app = QApplication(sys.argv)
mainView = MainView()
mainView.show()
Controller(view=mainView,)
app.exec()

qt.qpa.xcb: X server does not support XInput 2


0